In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

es_client.indices.create(
    index=index_name,
    body=index_settings
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

/Users/marco/Developer/miniconda3/envs/llm-zoomcamp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 505.14it/s]


In [18]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        # result_docs.append(hit['_score'])
    
    return result_docs

In [19]:
def search(query):
    search_results = elastic_search(query)
    return search_results

In [21]:
query = "How do I execute a command in a running docker container?"
search_results = search(query)
print(search_results[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


In [34]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ""

    for doc in search_results:
        context += context_template.format(
            question=doc['question'],
            text=doc['text']
        ) + "\n\n"
    
    prompt = prompt_template.format(
        question=query,
        context=context
    )
    
    return prompt

In [37]:
query = "How do I execute a command in a running docker container?"
search_results = search(query)
filtered_results = search_results[:3]
print(filtered_results)
prompt = build_prompt(query, filtered_results)
print(len(prompt))

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}, {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'm

In [40]:
import tiktoken

In [60]:
def token_count(text):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoding.encode(text)

    token_count = len(tokens)
    return(token_count)

In [51]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    

    prompt_template = """
You're a course teaching assistant. You're given the QUESTION from a student and some CONTEXT about the course.
Your task is to answer the QUESTION based on the provided CONTEXT. Use only the facts from the CONTEXT when answering the question.
The user does not know about the CONTEXT. Only you do, so don't mention the CONTEXT. Reply as if the knowledge comes from you.
If the CONTEXT does not contain the answer, respond with "I don't know."

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context = ""

    for doc in search_results:
        context += context_template.format(
            question=doc['question'],
            text=doc['text']
        ) + "\n\n"
    
    prompt = prompt_template.format(
        question=query,
        context=context
    )
    
    return prompt

In [52]:
import ollama

In [53]:
def llm(prompt):
    response = ollama.chat(model='llama3:8b-instruct-q5_K_M', messages=[
        {
            'role': 'user',
            'content': prompt,
        },
    ])
    return response['message']['content']

In [55]:
query = "How do I execute a command in a running docker container?"
search_results = search(query)
filtered_results = search_results[:3]
prompt = build_prompt(query, filtered_results)
response = llm(prompt)
print(response)

To execute a command in a running docker container, you would use the `docker exec` command. First, find the container ID using `docker ps`, then run `docker exec -it <container-id> bash`. This will open a new terminal session inside the running container.


In [58]:
input = 150
output = 250
requests = 1000
costs = requests * (input * 0.005 + output * 0.015)
input_tokens = 150  # tokens per request
output_tokens = 250  # tokens per request
requests = 1000

# Prices per 1K tokens
input_price = 0.005  # $ per 1K tokens
output_price = 0.015  # $ per 1K tokens

# Calculate total tokens
total_input_tokens = input_tokens * requests  # 150,000 tokens
total_output_tokens = output_tokens * requests  # 250,000 tokens

# Calculate costs
input_cost = (total_input_tokens / 1000) * input_price
output_cost = (total_output_tokens / 1000) * output_price

total_cost = input_cost + output_cost

print(f"Total cost: ${total_cost:.2f}")

Total cost: $4.50
